In [3]:
# df = spark.read.parquet('../../../../0_global_data/fd-cidacs-rl/sinthetic-dataset-A.parquet/')
# df.coalesce(1).write.csv('../../../../0_global_data/fd-cidacs-rl/sinthetic-dataset-A-legacy.csv', header=True)
# df.limit(2).toPandas()

,id_cidacs_a,nome_a,nome_mae_a,dt_nasc_a,sexo_a
0,1,YASMIM VITORIA MATIAS FONSECA,TACIANY DOS SANTOS,20071122,2
1,2,PEDRO HENRIQUE MARTINS DE CARVALHO,FRANCILEIDE DOS SANTOS ALVES,20061102,1


In [4]:
import sys
import json
from elasticsearch import Elasticsearch, helpers
import pandas as pd
import time

In [5]:
start = time.time()

In [6]:
pd.set_option("display.max_rows", 101)
pd.set_option("display.max_columns", 101)

In [7]:
year = 'total'

In [8]:
base_path = "../../../../0_global_data/fd-cidacs-rl/sinthetic-dataset-A-legacy.csv/part-00000-d2adadb0-df8d-43ee-a436-1c89e1afd697-c000.csv"
index = "fd-cidacs-rl-legacy"

In [9]:
# showing the live birth data
spark.read.csv(base_path, header=True).limit(10).toPandas()

,id_cidacs_a,nome_a,nome_mae_a,dt_nasc_a,sexo_a
0,1,YASMIM VITORIA MATIAS FONSECA,TACIANY DOS SANTOS,20071122,2
1,2,PEDRO HENRIQUE MARTINS DE CARVALHO,FRANCILEIDE DOS SANTOS ALVES,20061102,1
2,3,FABRICIO RODRIGUES DOS SANTOS,MARCELA MACHADO DA SILVA,20071107,1
3,4,VINICIUS DA SILVA SOUZA,ELIZANGELA LIMA DA SILVA,20071008,1
4,5,LUAN FERREIRA DO NASCIMENTO,KEZIA NUNES GALDINO MONTEIRO,20080128,1
5,6,JOAO VITOR BORGES PEREIRA,DICIMARA LIMA MONTEIRO,20061002,1
6,7,JOAO PEDRO BATISTA DOS SANTOS,SOILA COSTA DA SILVA,20070903,1
7,8,AMANDA BANDEIRA BARRA,EDINEIA ESQUERDO BRAGA,20070610,2
8,9,GABRIEL COUTO GOMES,ROSILDA LEAL BARBOSA,20061008,1
9,10,LUIZA VITORIA BATISTA DOS SANTOS,EDILZA MAGALHAES DE SOUZA MARTINS,20061027,2


In [10]:
spark.read.csv(base_path, header=True).count()

1000000

In [11]:
es = Elasticsearch()

In [12]:
bulksize = 12*1000*1000

In [13]:
h = {
    'id_cidacs_a':0,
    'nome_a':1,
    'nome_mae_a':2,
    'dt_nasc_a':3,
    'sexo_a':4  
}

In [16]:
f = open(base_path, 'r')
data  = list()
header = True
n = 0
for l in f:
    n += 1
    if header:
        header = False
        continue
    l = l.replace('\n', '').split(',')
    seq = l[h['id_cidacs_a']].strip()
    nome_a = l[h['nome_a']].strip()
    nome_mae_a = l[h['nome_mae_a']].strip()
    dt_nasc_a = l[h['dt_nasc_a']].strip()
    sexo_a = l[h['sexo_a']].strip()

    content = {
        '_index':index,
        '_type':'pessoa',
        '_id':seq,
        '_source':{
            'nome_a':nome_a,
            'nome_mae_a':nome_mae_a,
            'dt_nasc_a':dt_nasc_a,
            'sexo_a':sexo_a
        }
    }
    
    data.append(content)
    
    # commit after bulksize is reached
    if sys.getsizeof(data)*10 > bulksize:
        force = True
        while(force):
            try:
                helpers.bulk(es, data)
                force = False
            except:
                continue
        # clean data
        data = list()

helpers.bulk(es, data)
f.close()
print(n)

/home/pierre/anaconda3/lib/python3.8/site-packages/elasticsearch/connection/base.py:193: ElasticsearchDeprecationWarning: [types removal] Specifying types in bulk requests is deprecated.
  warnings.warn(message, category=ElasticsearchDeprecationWarning)


1000001


In [17]:
print("Tempo total: {} secs".format(time.time()-start))

Tempo total: 163.60894918441772 secs


In [18]:
len(data)

119506

In [19]:
n

1000001

In [20]:
len(data)

119506

In [21]:
es.count(index=index)

{'count': 1000000,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}}